# Obtain Embeddings From The FaceNet Model

In this notebook I load the FaceNet model to obtain embeddings from the images of cropped faces. I will use these embeddings to compare user faces to my data frame in my facial similarity system.

I will be using Hiroki Taniai's Keras implementation of David Sandberg's open source FaceNet TensorFlow model. The link to the GitHub can be found [here](https://github.com/nyoki-mtl/keras-facenet).



In [25]:
#Import the necessary libraries here

import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image

from tqdm import tqdm
from tqdm import tqdm_notebook

import ast

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import load_model

from os import path

In [2]:
#Load the FaceNet model and pretrained weights
facenet_model = load_model('FaceNet_Model/facenet_keras.h5')
facenet_model.load_weights('FaceNet_Model/facenet_keras_weights.h5')

print(facenet_model.inputs)
print(facenet_model.outputs)

#The model requires an input image to be reshaped to the size (160*160*3), where the 3
#represents RGB format. If the image is black and white it must be converted.

#The model will output a 128 vector facial embedding that represents the most important
#features of the face. I will use these embeddings to compare each face in my program

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[<tf.Tensor 'input_1:0' shape=(?, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/batchnorm/add_1:0' shape=(?, 128) dtype=float32>]


In [10]:
#Load the converted metadata files for the IMDB and Wikipedia images
photo_df = pd.read_csv('Photo_Dataframes/All_Photo_Data.csv')

In [11]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
1,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766
2,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1346607872...,[ 78.23569407 52.33512938 194.25423516 168.35...,3.479189
3,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1363385088...,[ 64.30867104 64.30867104 207.26768087 207.26...,3.870171
4,Lauren Bacall,1924-09-16,0.0,1974,50,image_data/imdb_data/02/nm0000002_rm221957120_...,[3173.14469259 401.04083657 4059.15207489 128...,4.096431


In [12]:
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214617 entries, 0 to 214616
Data columns (total 8 columns):
name              214617 non-null object
dob               214617 non-null object
gender            214617 non-null float64
photo_taken       214617 non-null int64
age_when_taken    214617 non-null int64
file_path         214617 non-null object
face_location     214617 non-null object
face_score        214617 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 13.1+ MB


# Obtain The Embeddings Of Each Picture

In this section I open each image of cropped faces, resize and reformat them to the model's specifications and save the embeddings vector as a numpy array in the metadata data frame.

In [5]:
#This function opens the file and obtains an array representation of the picture
#It converts the image to RGB format if the image is black and white

def obtain_image_pixels(filename):
    image = Image.open(filename)
    image = image.convert('RGB')
    return asarray(image)

In [6]:
#This function resizes the picture array to the model specifications (160*160*3)

def resize_picture(image_array, dimensions = (160,160)):
    face_array_resized = Image.fromarray(image_array)
    face_array_resized = face_array_resized.resize(dimensions)
    return asarray(face_array_resized)

In [7]:
#The following function is adapted from 'Deep Learning for Computer Vision' by
#Jason Brownlee, Page (508)

#This function takes in a file path and returns the face embeddings vector from the
#image. First the image is opened and stored as a matrix. Next the image matrix is
#resized. The numbers in the matrix are then converted into the float data type so
#they can store decimals. The individual numbers are then standardized by removing the
#mean of the pixel intensity and then dividing each pixel by the standard deviation
#A fourth dimension is then added to the image tensor which is used to keep track of
#set of samples the image belongs to (the category and set, i.e. training and testing)
#Finally the model.predict method returns the 128 element face embedding vector that
#corresponds to the image

def get_embedding(filename):
    #obtain the face pixels
    cropped_face_array = obtain_image_pixels(filename)
    resized_face = resize_picture(cropped_face_array)
    
    # standardize pixel values across channels
    resized_face = resized_face.astype('float32')
    mean, std = resized_face.mean(), resized_face.std()
    resized_face = (resized_face - mean) / std
    
    
    # transform face into one sample
    observation = expand_dims(resized_face, axis=0)
    # make prediction to get embedding
    yhat = facenet_model.predict(observation)
    return yhat[0]
    

# Obtain the Embeddings Array for Each Picture

I create a new column in the photo_df data frame called 'embeddings_fn' that stores the FaceNet model embeddings for each cropped face. These embeddings will be used to calculate the similarity between user faces and the data set of faces. I perform this step in batches of 50,000 rows to ensure I save my data. I then take each batch and apply the get_embedding() function to each row, after each batch is complete I save the data frame.

In [8]:
photo_df_a = photo_df[0:50000]
photo_df_b = photo_df[50000:100000]
photo_df_c = photo_df[100000:150000]
photo_df_d = photo_df[150000:200000]
photo_df_e = photo_df[200000:]


## Photo Dataframe With Embeddings - Part A

In [9]:
tqdm.pandas()
photo_df_a['embeddings_fn'] = photo_df_a.file_path.progress_apply(lambda x: get_embedding(x))

100%|██████████| 50000/50000 [1:19:12<00:00, 10.52it/s]     


In [10]:
photo_df_a.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_a.csv', index = False)

## Photo Dataframe With Embeddings - Part B

In [20]:
tqdm.pandas()
photo_df_b['embeddings_fn'] = photo_df_b.file_path.progress_apply(lambda x: get_embedding(x))

100%|██████████| 50000/50000 [1:07:18<00:00, 12.38it/s]


In [21]:
photo_df_b.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_b.csv', index = False)

## Photo Dataframe With Embeddings - Part C

In [22]:
tqdm.pandas()
photo_df_c['embeddings_fn'] = photo_df_c.file_path.progress_apply(lambda x: get_embedding(x))

100%|██████████| 50000/50000 [1:09:04<00:00, 11.91it/s]


In [23]:
photo_df_c.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_c.csv', index = False)

## Photo Dataframe With Embeddings - Part D

In [24]:
tqdm.pandas()
photo_df_d['embeddings_fn'] = photo_df_d.file_path.progress_apply(lambda x: get_embedding(x))

100%|██████████| 50000/50000 [1:15:48<00:00, 11.59it/s]


In [25]:
photo_df_d.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_d.csv', index = False)

## Photo Dataframe With Embeddings - Part E

In [9]:
tqdm.pandas()
photo_df_e['embeddings_fn'] = photo_df_e.file_path.progress_apply(lambda x: get_embedding(x))

100%|██████████| 50000/50000 [1:11:04<00:00, 11.72it/s]


In [10]:
photo_df_e.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_e.csv', index = False)

# Testing The Loaded Dataframes

Here I load each saved dataframe and test the 'embeddings_fn column to see if I am able to utilize the embeddings vector that I stored.

In [17]:
#In a now defunct notebook I eliminated one image because its facial embeddings vector
#contained only nan values. This is because the image itself was a solid white box, it was
#not stored properly in the original web scrape. This is why i removed this image. Its row
#id corresponds to a row and data frame that was removed as I lowered the second face
#score threshold from 0.9 to 0.25.

#photo_df_h.drop(photo_df_h.index[7428], inplace = True)

In [18]:
#Load the saved data frames with the FaceNet embeddings

photo_df_a = pd.read_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_a.csv')
photo_df_b = pd.read_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_b.csv')
photo_df_c = pd.read_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_c.csv')
photo_df_d = pd.read_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_d.csv')
photo_df_e = pd.read_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_e.csv')

In [19]:
#Concatenate the dataframes
photo_df = pd.concat([photo_df_a, photo_df_b, photo_df_c, photo_df_d, photo_df_e]).reset_index(drop=True)

photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,embeddings_fn
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117,[ 0.1044133 -0.80462444 -0.86017245 -0.593098...
1,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766,[-1.3740315 -0.807325 -0.21413553 0.019938...
2,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1346607872...,[ 78.23569407 52.33512938 194.25423516 168.35...,3.479189,[-0.4142363 0.04628806 -0.04605452 -1.316444...
3,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1363385088...,[ 64.30867104 64.30867104 207.26768087 207.26...,3.870171,[-1.84994042e-01 7.11622715e-01 4.40206051e-...
4,Lauren Bacall,1924-09-16,0.0,1974,50,image_data/imdb_data/02/nm0000002_rm221957120_...,[3173.14469259 401.04083657 4059.15207489 128...,4.096431,[-0.7291794 0.68375134 2.1871135 -1.996834...


In [21]:
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214616 entries, 0 to 214615
Data columns (total 9 columns):
name              214616 non-null object
dob               214616 non-null object
gender            214616 non-null float64
photo_taken       214616 non-null int64
age_when_taken    214616 non-null int64
file_path         214616 non-null object
face_location     214616 non-null object
face_score        214616 non-null float64
embeddings_fn     214616 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 14.7+ MB


In [22]:
#The embeddings vectors were converted to strings in the .csv format. I create
#a function below to convert them back to numpy arrays so I can use them later in
#my analysis

print(type(photo_df.embeddings_fn[0]))

<class 'str'>


In [26]:
#The following function converts the string that represents the face embeddings into
#a numpy vector

def convert_csv_to_embeddings(embedding_string):
    
    #I replace the '\n' and spaces in descending sequential order (i.e. 5 spaces to 1,
    #4 spaces to 1, etc.)
    embedding_string = embedding_string.replace('\n', '').replace('     ', ' ').replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace('[ ', '[').replace(' ]', ']').replace(' ', ', ')
    
    #This code returns the string as an array in the proper type
    return asarray(ast.literal_eval(embedding_string)).astype('float32')

photo_df.embeddings_fn = photo_df.embeddings_fn.apply(lambda x: convert_csv_to_embeddings(x))
print('The dataframe is now loaded.\n')


The dataframe is now loaded.



In [27]:
#The embeddings vectors were successfully converted back to numpy arrays after being
#stored as values in a pandas data frame in .csv format

print(type(photo_df.embeddings_fn[0]))

<class 'numpy.ndarray'>
